# Initiez-vous au MLOps (partie 1/2)
## Option B - Exercice - Élaborez un modèle de scoring

Vous êtes Data Scientist au sein d'une société financière, nommée "Prêt à dépenser", qui propose des crédits à la consommation pour des personnes ayant peu ou pas du tout d'historique de prêt.
 
L’entreprise souhaite mettre en œuvre un outil de “scoring crédit” pour calculer la probabilité qu’un client rembourse son crédit, puis classifie la demande en crédit accordé ou refusé. Elle souhaite donc développer un algorithme de classification en s’appuyant sur des sources de données variées (données comportementales, données provenant d'autres institutions financières, etc.)

La mission :
- Construire et optimiser un modèle de scoring qui donnera une prédiction sur la probabilité de faillite d'un client de façon automatique.
- Analyser les features qui contribuent le plus au modèle, d’une manière générale (feature importance globale) et au niveau d’un client (feature importance locale), afin, dans un soucis de transparence, de permettre à un chargé d’études de mieux comprendre le score attribué par le modèle.
- Mettre en œuvre une approche globale MLOps de bout en bout, du tracking des expérimentations à la pré-production du modèle.

Mise en oeuvre des étapes orientées MLOps suivantes :
- Dans le notebook d’entraînement des modèles, générer à l’aide de MLFlow un tracking d'expérimentations.
- Lancer l’interface web “UI MLFlow" d'affichage des résultats du tracking.
- Réaliser avec MLFlow un stockage centralisé des modèles dans un “model registry”.
- Tester le serving MLFlow.

### Objectif de cette étape :
* Un ou plusieurs modèles entraînés, avec validation croisée et premières métriques d’évaluation.
#### A faire :
* Commencer par tester des modèles simples (Logistic Regression, Random Forest).
* Comparer ensuite avec des modèles plus puissants (XGBoost, LightGBM).
* UtiliserStratifiedKFoldpour conserver la distribution de classes et pour garantir une évaluation robuste.
* Entraîner les modèles dans des notebooks clairs et documentés.
* Stocker les scores et les hyperparamètres testés.
#### Utilisation des métriques suivantes :
* AUC-ROC
* Recall sur la classe minoritaire
* F1-score
* Coût métier personnalisé (FN > FP)
#### Réalisation : Même notebook que le notebook 3 mais en prenant comme scoring le coût métier dans le GridSearch
**L'ensemble des étapes est suivi et enregistré sur un MLFLow en local**
1) Séparation du jeu et création de la fonction de preprocessing et pipeline
2) Mise en place de la recherche d'hyperparamètres via GridSearchCV
    * Logistic Regression
    * Random Forest
    * XGBoost
    * LightGBM
    * Chaque modèle est enregistrée dans un run différent afin de comparer les scores de la métrique principale AUC
    * Etant donné le long traitement des modèles, nous avons utilisé que class weight pour gérer les déséquilibres, 
    en appliquant "balanced" quand c'était possible sans chercher d'autres combinaisons de class weight.
    * Dans la compréhension des résultats, mise en avant de l'importance de la métrique recall de la classe 1 afin de prendre le + possible de "mauvais payeurs'.
3) Validation croisée avec les meilleurs paramètres sur l'ensemble des modèles
    * cross_validate couplé avec cross_validate_predict pour observer les résultats des métriques suivantes par modèle :
        * roc_auc
        * precision
        * recall
        * f1
        * average_precision
        * balanced_accuracy
    * Calcul du seuil métier : avec un coût métier de l'ordre de 10 par FN et 1 par FP
    * Graphique de la courbe ROC-AUC
    * Graphique de comparaison du seuil métier versus le seuil de base (0.5)
    * Enregistrement des résultats sous MLFlow dans un autre run mais dans une version 2 de chaque model dans model registry
4) Version des résultats du modèle avec le seuil métier comme référence
    * Enregistrement des résultats sous MLFlow dans un autre run mais dans une version 3 de chaque model dans model registry
5) Comparaison du meilleur modèle sous MLFlow
6) Vérification de cohérence avec les consignes, aucun des scores AUC ne dépassent 0.82, ils se situent entre 0.75 & 0.78

### Importation des librairies

In [1]:
# Librairies de base
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Librairies pour sauvegarde
import joblib
import os

# Ajout de MlFlow
import mlflow
import mlflow.sklearn
mlflow.set_tracking_uri("file://" + os.path.abspath("../mlruns"))
from mlflow.tracking import MlflowClient

# Libraires scikit-learn
from sklearn.impute import SimpleImputer
from sklearn.model_selection import (train_test_split,GridSearchCV, cross_validate,cross_val_predict, StratifiedKFold)
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import average_precision_score, precision_recall_curve, roc_auc_score, auc, f1_score, balanced_accuracy_score, make_scorer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.neural_network import MLPClassifier
from mlflow.models import infer_signature
from sklearn.metrics import roc_curve, roc_auc_score

from xgboost import XGBClassifier
import lightgbm as lgb
import shap

# Meilleure visualisation des grands df
import itables.options as opt
from itables import show

# Activation de l’affichage interactif automatiquement dans les notebooks
opt.warn_on_undocumented_option = False
opt.notebook_connected = True
opt.maxBytes = 0 # Pour ne pas tronquer les gros DataFrames
opt.maxColumns = 0

import warnings
warnings.filterwarnings(
    "ignore",
    message=".*does not have valid feature names.*"
)

### Chargement du dataset fusionné

In [2]:
app_train_modelisation = pd.read_csv("../data/processed/app_train_modelisation.csv")

#### Séparation de notre jeu de données

In [3]:
# Séparation de notre jeu fusionné en 2 grâce à la distinction - technique utilisée dans le kernel de Kaggle
train_df = app_train_modelisation[app_train_modelisation['TARGET'].notnull()].copy() # jeu qui sera utilisé pour notre projet
test_df = app_train_modelisation[app_train_modelisation['TARGET'].isnull()].copy()

# Mise en place de nos X et y
y = train_df['TARGET']
X = train_df.drop(columns=['TARGET', 'is_train'], errors='ignore')
X_test_kaggle = test_df.drop(columns=['TARGET', 'is_train'], errors='ignore') # pour Kaggle (pas utilisé ici)

cols_encode = [col for col in X.columns if X[col].nunique() <= 2]
cols_num = [col for col in X.columns if X[col].nunique()>2 ]

#### On met en place notre suivi MLFlow

In [4]:
mlflow.set_experiment("Tracking_models")

<Experiment: artifact_location='file:///Users/florianschorer/Documents/OpenClassrooms/Projets/OC_P6/Modele_scoring_MLFlow/mlruns/321322968827569333', creation_time=1762262527508, experiment_id='321322968827569333', last_update_time=1762262527508, lifecycle_stage='active', name='Tracking_models', tags={'mlflow.experimentKind': 'custom_model_development'}>

#### Notre jeu étant déjà séparé, on peut déjà créer notre fonction pour le preprocessing

In [5]:
def save_preprocessor(cols_num,cols_encode):
    preprocessor = ColumnTransformer(
        transformers = [
        ('num', StandardScaler(),cols_num),
        ('cat','passthrough',cols_encode)
        ])
    joblib.dump((preprocessor), "../models/preprocessor.pkl")
    return preprocessor

#### Création de la fonction du pipeline

In [7]:
def save_pipeline(preprocessor,model):
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    joblib.dump((pipeline), "../models/pipeline.pkl")
    return pipeline

#### Création de la fonction du score métier

In [8]:
def business_cost(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return 10 * fn + fp
business_cost_scorer = make_scorer(business_cost, greater_is_better=False) # Afin de miniser le coût

#### Création de la fonction du GridSearchCV

In [9]:
def gridsearchcv(pipeline, param_grid, X, y):
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    grid_search = GridSearchCV(
        estimator = pipeline,
        param_grid = param_grid,
        cv = cv,
        n_jobs = -1,
        scoring = business_cost_scorer,
        verbose = 0
    )
    # Entraînement du modèle
    grid_search.fit(X,y)
    # J'enregistre quel est le meilleur modèle, je réalise la prédiction et j'évalue
    best_model = grid_search.best_estimator_
    print("Meilleurs paramètres :", grid_search.best_params_)
    print("Meilleur_cout_metier", grid_search.best_score_)
    return grid_search

#### Modèle LogisticRegression
* Vu la taille du fichier, on ne fait tourner que quelques hyperparamètres

In [10]:
preprocessor = save_preprocessor(cols_num,cols_encode)
model = LogisticRegression(max_iter=1000,random_state=42)
pipeline = save_pipeline(preprocessor,model)
param_grid = {
    'model__penalty': ['l2'],'model__C': [0.5, 1.0, 2.0],
    'model__class_weight': ['balanced']}
grid_search = gridsearchcv(pipeline, param_grid, X, y)

Meilleurs paramètres : {'model__C': 0.5, 'model__class_weight': 'balanced', 'model__penalty': 'l2'}
Meilleur_cout_metier -52688.333333333336


* Nous avons ici un score AUC correcte étant donné notre jeu déséquilibré.
* On a utilisé ici le paramètre balanced afin gérér notre fort déséquilibre (92%-8%).
* Le résultat peut s'expliquer par l'application d'une bonne partie du feature engineering du kernel de Kaggle.
* Les meilleurs paramètres sont les suivants : **{'model__C': 0.5, 'model__class_weight': 'balanced', 'model__penalty': 'l2'}**

#### Enregistrement des paramètres et du modèle de Logistic Regression sur MLFlow

In [11]:
# Définition du clien pour gérer la partie model registry
client = MlflowClient()
projet_description = ("Nous effectuons une recherche des hyperparamètres pour un modèle de LogisticRegression via GridSearchCV avec le coût métier.")
with mlflow.start_run(run_name="LogisticRegression_GridSearch_cout_metier",
                      tags={"Training Info" : "LogisticRegression_GridSearch_cout_metier",
                            "mlflow.note.content" : projet_description
                            }):
   # Meilleur modèle issu du GridSearch
    best_model = grid_search.best_estimator_

   # Exemple du dataset, on retrouvera les colonnes et 3 lignes de données
    X_sample = X.head(3).copy()
    y_sample = best_model.predict(X.head(3))
    signature = infer_signature(X.head(3), y_sample)
    
    # Log des meilleurs hyperparamètres
    mlflow.log_params(grid_search.best_params_)

    # Log du meilleur score du coût métier
    mlflow.log_metric("Meilleur_cout_metier", grid_search.best_score_)

    # Log de tous les hyperparamètres testés
    tested_params = grid_search.param_grid
    flat_tested = {f"tested_{k}": str(v) for k, v in tested_params.items()}
    mlflow.log_params(flat_tested)

    # Log du pipeline complet dans MLflow
    mlflow.sklearn.log_model(
        best_model, name="LogisticRegression_GridSearch_cout_metier",
        registered_model_name="LogisticRegression_GridSearch_cout_metier",
        input_example=X_sample,
        signature=signature)
    
    client.set_model_version_tag(name="LogisticRegression_GridSearch_cout_metier", version=1, key="LogisticRegression_GridSearch_cout_metier", value="...")
    client.update_model_version(
    name="LogisticRegression_GridSearch_cout_metier",
    version=1,
    description="Recherche d'hyperparamètres via GridSearchCV pour LogisticRegression avec le coût métier"
)


python(90807) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(90808) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
/Users/florianschorer/Library/Caches/pypoetry/virtualenvs/modele-scoring-mlflow-N-Hr5ii3-py3.13/lib/python3.13/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ fo

#### Modèle Random Forest
* Réutilisation des fonctions, on change juste le modèle et les paramètres testés

In [10]:
preprocessor = save_preprocessor(cols_num,cols_encode)
model = RandomForestClassifier(random_state=42)
pipeline = save_pipeline(preprocessor,model)
param_grid = {
    'model__class_weight': ['balanced'],
    'model__n_estimators': [100, 200],
    'model__max_depth': [12],
    'model__min_samples_leaf': [1, 4]}
grid_search = gridsearchcv(pipeline, param_grid, X, y)

Meilleurs paramètres : {'model__class_weight': 'balanced', 'model__max_depth': 12, 'model__min_samples_leaf': 4, 'model__n_estimators': 200}
Meilleur_cout_metier -57367.333333333336


* Nous avons ici un score AUC correcte étant donné notre jeu déséquilibré.
* On a utilisé ici le paramètre balanced afin gérér notre fort déséquilibre (92%-8%).
* Le résultat peut s'expliquer par l'application d'une bonne partie du feature engineering du kernel de Kaggle.
* Le score de ROC AUC est légèrement moins que notre baseline, le modèle de Logistic Regression.
* Les meilleurs paramètres sont les suivants : **{'model__class_weight': 'balanced', 'model__max_depth': 12, 'model__min_samples_leaf': 4, 'model__n_estimators': 200}**

#### Enregistrement des paramètres et du modèle de Random Forest sur MLFlow

In [11]:
# Définition du clien pour gérer la partie model registry
client = MlflowClient()
projet_description = ("Nous effectuons une recherche des hyperparamètres pour un modèle de Random Forest via GridSearchCV avec le coût métier.")
with mlflow.start_run(run_name="RandomForest_GridSearch_cout_metier",
                      tags={"Training Info" : "RandomForest_GridSearch_cout_metier",
                            "mlflow.note.content" : projet_description
                            }):
   # Meilleur modèle issu du GridSearch
    best_model = grid_search.best_estimator_

   # Exemple du dataset, on retrouvera les colonnes et 3 lignes de données
    X_sample = X.head(3).copy()
    y_sample = best_model.predict(X.head(3))
    signature = infer_signature(X.head(3), y_sample)
    
    # Log des meilleurs hyperparamètres
    mlflow.log_params(grid_search.best_params_)

    # Log du meilleur score moyen ROC AUC
    mlflow.log_metric("Meilleur_cout_metier", grid_search.best_score_)

    # Log de tous les hyperparamètres testés
    tested_params = grid_search.param_grid
    flat_tested = {f"tested_{k}": str(v) for k, v in tested_params.items()}
    mlflow.log_params(flat_tested)

    # Log du pipeline complet dans MLflow
    mlflow.sklearn.log_model(
        best_model, name="RandomForest_GridSearch_cout_metier",
        registered_model_name="RandomForest_GridSearch_cout_metier",
        input_example=X_sample,
        signature=signature)
    
    client.set_model_version_tag(name="RandomForest_GridSearch_cout_metier", version=1, key="RandomForest_GridSearch_cout_metier", value="...")
    client.update_model_version(
    name="RandomForest_GridSearch_cout_metier",
    version=1,
    description="Recherche d'hyperparamètres via GridSearchCV pour RandomForest avec le coût métier"
)

/Users/florianschorer/Library/Caches/pypoetry/virtualenvs/modele-scoring-mlflow-N-Hr5ii3-py3.13/lib/python3.13/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
Successfully registered model 'RandomForest_GridSearch_cout_metier'.
Created version '1' of model 'RandomForest_GridSearch_cout_metier'.


#### Modèle XGBoost Classifier

In [13]:
preprocessor = save_preprocessor(cols_num,cols_encode)
model = XGBClassifier(eval_metric='logloss',random_state=42)
pipeline = save_pipeline(preprocessor,model)
param_grid = {
    'model__n_estimators': [100, 300],            
    'model__max_depth': [3, 4, 6],              
    'model__learning_rate': [0.05, 0.08],    
    'model__subsample': [0.8],               
    'model__scale_pos_weight': [12, 14]       
    }
grid_search = gridsearchcv(pipeline, param_grid, X, y)

Meilleurs paramètres : {'model__learning_rate': 0.08, 'model__max_depth': 4, 'model__n_estimators': 300, 'model__scale_pos_weight': 12, 'model__subsample': 0.8}
Meilleur_cout_metier -51034.333333333336


* Nous avons ici un score AUC correcte étant donné notre jeu déséquilibré.
* On a utilisé ici le paramètre balanced afin gérér notre fort déséquilibre (92%-8%).
* Le résultat peut s'expliquer par l'application d'une bonne partie du feature engineering du kernel de Kaggle.
* Le score de ROC AUC est meilleur que les deux modèles précédents.
* Les meilleurs paramètres sont les suivants : **{'model__learning_rate': 0.08, 'model__max_depth': 4, 'model__n_estimators': 300, 'model__scale_pos_weight': 12, 'model__subsample': 0.8}**

#### Enregistrement des paramètres et du modèle de XGBoost sur MLFlow

In [14]:
# Définition du clien pour gérer la partie model registry
client = MlflowClient()
projet_description = ("Nous effectuons une recherche des hyperparamètres pour un modèle de XGBoost via GridSearchCV avec le coût métier.")
with mlflow.start_run(run_name="XGBoost_GridSearch_cout_metier",
                      tags={"Training Info" : "XGBoost_GridSearch_cout_metier",
                            "mlflow.note.content" : projet_description
                            }):
   # Meilleur modèle issu du GridSearch
    best_model = grid_search.best_estimator_

   # Exemple du dataset, on retrouvera les colonnes et 3 lignes de données
    X_sample = X.head(3).copy()
    y_sample = best_model.predict(X.head(3))
    signature = infer_signature(X.head(3), y_sample)
    
    # Log des meilleurs hyperparamètres
    mlflow.log_params(grid_search.best_params_)

    # Log du meilleur score moyen ROC AUC
    mlflow.log_metric("Meilleur_cout_metier", grid_search.best_score_)

    # Log de tous les hyperparamètres testés
    tested_params = grid_search.param_grid
    flat_tested = {f"tested_{k}": str(v) for k, v in tested_params.items()}
    mlflow.log_params(flat_tested)

    # Log du pipeline complet dans MLflow
    mlflow.sklearn.log_model(
        best_model, name="XGBoost_GridSearch_cout_metier",
        registered_model_name="XGBoost_GridSearch_cout_metier",
        input_example=X_sample,
        signature=signature)
    
    client.set_model_version_tag(name="XGBoost_GridSearch_cout_metier", version=1, key="XGBoost_GridSearch_cout_metier", value="...")
    client.update_model_version(
    name="XGBoost_GridSearch_cout_metier",
    version=1,
    description="Recherche d'hyperparamètres via GridSearchCV pour XGBoost avec le coût métier"
)

/Users/florianschorer/Library/Caches/pypoetry/virtualenvs/modele-scoring-mlflow-N-Hr5ii3-py3.13/lib/python3.13/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
Successfully registered model 'XGBoost_GridSearch_cout_metier'.
Created version '1' of model 'XGBoost_GridSearch_cout_metier'.


#### Modèle LGBMClassifier

In [15]:
preprocessor = save_preprocessor(cols_num,cols_encode)
model = lgb.LGBMClassifier(random_state=42)
pipeline = save_pipeline(preprocessor,model)
param_grid = {
    'model__n_estimators': [100, 300],
    'model__num_leaves': [30, 60],
    'model__learning_rate': [0.05, 0.1],
    'model__feature_fraction': [0.8],
    'model__bagging_fraction': [0.8],
    'model__scale_pos_weight': [12, 16],
    'model__verbosity': [-1],
    'model__force_row_wise': [True]}
grid_search = gridsearchcv(pipeline, param_grid, X, y)

Meilleurs paramètres : {'model__bagging_fraction': 0.8, 'model__feature_fraction': 0.8, 'model__force_row_wise': True, 'model__learning_rate': 0.05, 'model__n_estimators': 300, 'model__num_leaves': 60, 'model__scale_pos_weight': 12, 'model__verbosity': -1}
Meilleur_cout_metier -50906.333333333336


* Nous avons ici un score AUC correcte étant donné notre jeu déséquilibré.
* On a utilisé ici le paramètre balanced afin gérér notre fort déséquilibre (92%-8%).
* Le résultat peut s'expliquer par l'application d'une bonne partie du feature engineering du kernel de Kaggle.
* Le score de ROC AUC est meilleur que les trois modèles précédents, il surclasse de très peu le modèle d'XGBoost.
* Les meilleurs paramètres sont les suivants : **{'model__bagging_fraction': 0.8, 'model__feature_fraction': 0.8, 'model__learning_rate': 0.05, 'model__n_estimators': 300, 'model__num_leaves': 60, 'model__scale_pos_weight': 12}**

#### Enregistrement des paramètres et du modèle de LightGBM sur MLFlow

In [16]:
# Définition du clien pour gérer la partie model registry
client = MlflowClient()
projet_description = ("Nous effectuons une recherche des hyperparamètres pour un modèle de LightGBM via GridSearchCV avec le coût métier.")
with mlflow.start_run(run_name="LightGBM_GridSearch_cout_metier",
                      tags={"Training Info" : "LightGBM_GridSearch_cout_metier",
                            "mlflow.note.content" : projet_description
                            }):
   # Meilleur modèle issu du GridSearch
    best_model = grid_search.best_estimator_

   # Exemple du dataset, on retrouvera les colonnes et 3 lignes de données
    X_sample = X.head(3).copy()
    y_sample = best_model.predict(X.head(3))
    signature = infer_signature(X.head(3), y_sample)
    
    # Log des meilleurs hyperparamètres
    mlflow.log_params(grid_search.best_params_)

    # Log du meilleur score moyen ROC AUC
    mlflow.log_metric("Meilleur_cout_metier", grid_search.best_score_)

    # Log de tous les hyperparamètres testés
    tested_params = grid_search.param_grid
    flat_tested = {f"tested_{k}": str(v) for k, v in tested_params.items()}
    mlflow.log_params(flat_tested)

    # Log du pipeline complet dans MLflow
    mlflow.sklearn.log_model(
        best_model, name="LightGBM_GridSearch_cout_metier",
        registered_model_name="LightGBM_GridSearch_cout_metier",
        input_example=X_sample,
        signature=signature)
    
    client.set_model_version_tag(name="LightGBM_GridSearch_cout_metier", version=1, key="LightGBM_GridSearch_cout_metier", value="...")
    client.update_model_version(
    name="LightGBM_GridSearch_cout_metier",
    version=1,
    description="Recherche d'hyperparamètres via GridSearchCV pour LightGBM avec le coût métier"
)

/Users/florianschorer/Library/Caches/pypoetry/virtualenvs/modele-scoring-mlflow-N-Hr5ii3-py3.13/lib/python3.13/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
Successfully registered model 'LightGBM_GridSearch_cout_metier'.
Created version '1' of model 'LightGBM_GridSearch_cout_metier'.


#### Nous n'effectuons pas plus d'analyse ici car causé par le nombre d'hyperparamètres limités, les meilleurs paramètres, même en optimisant selon le coût métier ne change pas les hyperparamètres déjà en place dans le précédent notebook.